<a href="https://colab.research.google.com/github/AriannaBi/SimpleFCNN/blob/main/SimpleFCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import csv
import glob
from itertools import chain
from keras import models
from keras import layers


from keras.models import Sequential
from keras.layers import Dense


from collections import Counter

Function to read the name of the files

In [69]:
def type_file(name):
    name_file = name.split('/')[5][:3]
    print(name_file)
    if name_file == 'EDA':
        return 0, 0
    elif name_file == 'BVP':
        return '250ms', 1
    elif name_file == 'ACC':
        return '250ms', 2
    elif name_file == 'TEM':
        return 0, 5
    return -1

Function that given a list it returns the most frequence element in it. 

If it's 0,1,0,1 it returns 0 becuase it's the first most frequent element. 

If it's 1,0,1,0 it returns 1 for the same reason. 

In [70]:
def most_frequent(List):
	occurence_count = Counter(List)
	return occurence_count.most_common(1)[0][0]

# listt = [0,1,0,1]
# print(most_frequent(listt))

Function that creates the train set and the train label.

Train set is a 3D tensor where the dimensions are: 

x = 2400 cols

y = df.shape[0]//2400 rows

z = 6 types of values

In [75]:
def create_train_set_train_label(files):
    leng = 0
    for name_file in files:
        table = pd.read_parquet(name_file, engine='pyarrow')
        df = pd.DataFrame(table)
        timestamp_col = pd.to_datetime(df['timestamp'], unit='s')
        df['timestamp'] = timestamp_col
        arr_type = type_file(name_file)
        if arr_type == -1:
            print("Error in the file name")
            return -1
        if arr_type[0] != 0:
            df = df.resample(arr_type[0], on='timestamp').mean()
            df = df.reset_index()
        leng += df.shape[0]//2400 + int(df.shape[0] % 2400 > 0)
    
    train_set = np.zeros((6, leng, 2400),dtype=np.float64)
    train_label = np.zeros((6,leng, 1),dtype=np.uint8)
    # print(leng)
    return train_set, train_label


Read data to:

- Create the tensor
- Create train_set
- Create train_label
- Create test_set
- Create network 

In [108]:
def read_data(files):
    data = create_train_set_train_label(files)
    train_set = data[0]
    train_label = data[1]

    # READ ALL THE FILES AND FILL THE TENSOR IN THE CORRESPONDING PLACE
    # array of indices of row to fill
    # EDA, BVP, ACCX, ACCY, ACCZ, TEMP
    arr_idx = [0, 0, 0, 0, 0, 0]
    for name_file in files:
      # DataFrame
      table = pd.read_parquet(name_file, engine='pyarrow')
      # creating DataFrame
      df = pd.DataFrame(table)
      # converting timestamp
      timestamp_col = pd.to_datetime(df['timestamp'], unit='s')
      df['timestamp'] = timestamp_col
      # get if file it's among EDA, BVP, ACC, ST, otherwise return error
      arr_type = type_file(name_file)
      if arr_type == -1:
          print("Error in the file name")
          return -1

      # RESAMPLE the tensor
      # 0 means it's already 4Hz, otherwise resample with '250ms'
      if arr_type[0] != 0:
          df = df.resample(arr_type[0], on='timestamp').mean()
          df = df.reset_index()



      # FILL the tensor
      # number_rows = df.shape[0]//2400 + int(df.shape[0] % 2400 > 0)
      number_rows = df.shape[0]//2400
      sleep_window = []
      # print(number_rows, df.shape[0])
      
      # if not ACC, fill the #st matrix
      if arr_type[1] != 2:
          position_element_in_column = 0
          row_to_fill = arr_idx[arr_type[1]]

          for j in range(row_to_fill, number_rows):
            # everytime it enter the loop add one row
              arr_idx[arr_type[1]] += 1
              sleep_window.clear()
              for k in range(2400):
                  train_set[arr_type[1]][j][k] = df['value'][position_element_in_column]
                  sleep_window.append(df['Sleep'][position_element_in_column])
                  position_element_in_column += 1
              train_label[arr_type[1]][j][0] = most_frequent(sleep_window)
                  

      # otherwise fill ACC_X, ACC_Y, ACC_Z
      elif arr_type[1] == 2:
          list_axis_ACC = ['X', 'Y', 'Z']
          for idx, axis in enumerate(list_axis_ACC):
              index = 2 + idx
              row_to_fill = arr_idx[index]
              position_element_in_column = 0
              for j in range(row_to_fill, number_rows):
                  sleep_window.clear()
                  for k in range(2400):
                      train_set[index][j][k] = df[axis][position_element_in_column]
                      sleep_window.append(df['Sleep'][position_element_in_column])
                      position_element_in_column += 1
                  train_label[index][j][0] = most_frequent(sleep_window)
                    
          
    # print(train_set)
    # print(train_label)
    # for elem in train_set[1]:
    #     print(elem)

    # for i in range(0, len(train_set[1])):
    #   for j in range(0, len(train_set[1][i])):
    #     print(train_set[1][i][j])

    model = Sequential()
    # print(train_set[1])
    # column, input shape = third dimension z
    model.add(Dense(6, activation='relu', input_shape=(len(train_set[1]), 2400, ) , name="layer1"))
    # model.add(Dense(16, activation='relu', name="layer2"))
    # model.add(Dense(1, activation='sigmoid', name="layer3"))

    # opt = keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

    model.fit(train_set, train_label, epochs=6, batch_size=128)

    # test_loss, test_acc = model.evaluate(test, test_labels)
    # print('test_acc:', test_acc)

def read_folder_create_list():
    list_file = []
    list_file2 = glob.glob("./drive/MyDrive/Sessions/S02/*.parquet")
    # print(list_file2)
    list_file3 = glob.glob("./drive/MyDrive/Sessions/S03/*.parquet")
    # print(list_file3)
    list_file = glob.glob("./drive/MyDrive/Sessions/S04/*.parquet")

    for i in list_file3:
        list_file2.append(i)
    print(list_file2)


    # list_file = ['./drive/MyDrive/ACC_22.parquet']
    
    # , './drive/MyDrive/BVP_41.parquet', './drive/MyDrive/BVP_26.parquet']
    # , './drive/MyDrive/TEMP_25.parquet']
    read_data(list_file2)


read_folder_create_list()



['./drive/MyDrive/Sessions/S02/ACC_01.parquet', './drive/MyDrive/Sessions/S02/ACC_02.parquet', './drive/MyDrive/Sessions/S02/ACC_03.parquet', './drive/MyDrive/Sessions/S02/ACC_06.parquet', './drive/MyDrive/Sessions/S02/ACC_05.parquet', './drive/MyDrive/Sessions/S02/ACC_07.parquet', './drive/MyDrive/Sessions/S02/ACC_08.parquet', './drive/MyDrive/Sessions/S02/ACC_09.parquet', './drive/MyDrive/Sessions/S02/ACC_12.parquet', './drive/MyDrive/Sessions/S02/ACC_10.parquet', './drive/MyDrive/Sessions/S02/ACC_14.parquet', './drive/MyDrive/Sessions/S02/ACC_13.parquet', './drive/MyDrive/Sessions/S02/ACC_15.parquet', './drive/MyDrive/Sessions/S02/ACC_16.parquet', './drive/MyDrive/Sessions/S02/ACC_17.parquet', './drive/MyDrive/Sessions/S02/ACC_18.parquet', './drive/MyDrive/Sessions/S02/ACC_19.parquet', './drive/MyDrive/Sessions/S02/ACC_21.parquet', './drive/MyDrive/Sessions/S02/ACC_20.parquet', './drive/MyDrive/Sessions/S02/ACC_22.parquet', './drive/MyDrive/Sessions/S02/ACC_24.parquet', './drive/MyD